In [31]:
from dotenv import load_dotenv

import openai
import os
load_dotenv()

import logging
import sys
import os

import numexpr as ne

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
openai.api_key = os.getenv('OPENAI_API_KEY')


os.environ['NUMEXPR_MAX_THREADS'] = '4'
os.environ['NUMEXPR_NUM_THREADS'] = '2'

In [32]:
from llama_index import (
    VectorStoreIndex, 
    SimpleDirectoryReader, 
    StorageContext, 
    load_index_from_storage,
)
from llama_index.prompts import PromptTemplate

try:
    storage_context = StorageContext.from_defaults(persist_dir='../storage/cache/andrew/law')
    index = load_index_from_storage(storage_context)
    print('loading from disk')
except:
    documents = SimpleDirectoryReader('law/').load_data()
    index = VectorStoreIndex.from_documents(documents)
    index.storage_context.persist(persist_dir='../storage/cache/andrew/law/')
    print('persisting to disk')

INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
loading from disk


In [33]:
documents

[Document(id_='27e65522-a51b-4c75-b390-abf56251df5f', embedding=None, metadata={'page_label': '1', 'file_name': 'luat-canh-tranh.pdf', 'file_path': 'law/luat-canh-tranh.pdf', 'file_type': 'application/pdf', 'file_size': 868602, 'creation_date': '2023-12-25', 'last_modified_date': '2023-12-25', 'last_accessed_date': '2023-12-25'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, hash='d893a8c04ee03873733c4d1fa867c08bceb4758d848bf1cd424b60c03c129c92', text='LUẬT CẠNH TRANH   Căn cứ Hiến pháp nước Cộng hòa xã hội chủ nghĩa Việt Nam; Quốc hội ban hành Luật Cạnh tranh. Chương I NHỮNG QUY ĐNNH CHUNG Điều 1. Phạm vi điều chỉnh Luật này quy định về hành vi hạn chế cạnh tranh, tập trung kinh tế gây tác động hoặc có khả năng gây tác động hạn chế cạnh tranh đến thị trường

In [34]:
text_qa_template_str = (
    "Bạn là trợ lý của Andrew Huberman, người có thể đọc các ghi chú podcast của Andrew Huberman.\n"
    "Luôn trả lời truy vấn chỉ bằng cách sử dụng thông tin ngữ cảnh được cung cấp, "
    "chứ không phải kiến ​​thức sẵn có.\n"
    "Một số quy tắc cần tuân theo:\n"
    "1. Không bao giờ tham khảo trực tiếp bối cảnh nhất định trong câu trả lời của bạn.\n"
    "2. Tránh những câu như 'Dựa trên ngữ cảnh, ...' hoặc "
    "'Thông tin ngữ cảnh ...' hoặc bất cứ điều gì cùng"
    "những dòng đó."
    "Thông tin bối cảnh dưới đây.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Trả lời câu hỏi:{query_str}\n"
)

text_qa_template = PromptTemplate(text_qa_template_str)

In [35]:
from llama_index.node_parser import SentenceWindowNodeParser


# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

In [36]:
sentence_nodes = node_parser.get_nodes_from_documents(documents)

In [37]:
from llama_index.embeddings import resolve_embed_model
from llama_index import VectorStoreIndex, ServiceContext
from llama_index.llms import OpenAI

# llm = OpenAI(model="gpt-3.5-turbo")
# service_context = ServiceContext.from_defaults(
#     llm=llm, embed_model=embed_model
# )

In [38]:
sentence_index = VectorStoreIndex(sentence_nodes)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "

In [39]:
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor

query_engine = sentence_index.as_query_engine(
    similarity_top_k=5,
    # the target key defaults to `window` to match the node_parser's default
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
    ext_qa_template = text_qa_template
)

In [56]:
window_response = query_engine.query(
    "Giải quyết tranh chấp hoạt động hoá chất?"
)
print(window_response)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Giải quyết tranh chấp trong hoạt động hóa chất có thể được thực hiện thông qua các hình thức như thương lượng giữa các bên, hòa giải giữa các bên do một tổ chức hoặc cá nhân được các bên thỏa thuận làm trung gian hòa giải, và các hình thức khác.


In [57]:
for i in window_response.source_nodes:
    print(i.metadata['file_path'])
    print(i.score)
    print(i.text)
    print("="*130)

law/luat-hoa-chat.pdf
0.9041223633488765
2.  Cá nhân lợi dụng chức vụ, quyền hạn cửa quyền, hách dịch, gây khó khăn, phiền hà cho tổ chức, cá nhân hoạt động hóa chất; bao che cho người vi phạm pháp luật về hoạt động hóa chất hoặc thiếu trách nhiệm để xảy ra ô nhiễm, sự cố hóa chất thì tùy theo tính chất, mức độ vi phạm mà bị xử lý kỷ luật hoặc bị truy cứu trách nhiệm hình sự; nếu gây thiệt hại thì phải bồi thường theo quy định của pháp luật.  Điều 68.  Giải quyết tranh chấp trong hoạt động hóa chất Giải quyết tranh chấp trong hoạt động hóa chất được thực hiện bằng những hình thức sau đây: 1.  Thương lượng giữa các bên; 2.  Hòa giải giữa các bên do một tổ chức hoặc cá nhân được các bên thỏa thuận làm trung gian hòa giải; 3. 
law/luat-hoa-chat.pdf
0.8624302367117218
2.  Chủ đầu tư dự án hoạt động hóa chất không thuộc Danh mục quy định tại khoản 1 Điều 38 của Luật này phải xây dựng Biện pháp phòng ngừa, ứng phó sự cố hóa chất phù hợp với quy mô, điều kiện sản xuất và đặc tính của hóa chất